<a href="https://colab.research.google.com/github/DucDTran/AIO-Exercise/blob/feature%2Fmodule-2-project_image_retrieval/Project_Image_Depth_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np

In [2]:
!gdown -q 14gf8bcym_lTcvjZQmg8kwq3aXkENBxMQ
!gdown -q 1wxmiUdqMciuTOs0ouKEISl8-iTVXdOWn

In [3]:
!unzip -q 'Aloe_images.zip'
!unzip -q 'tsukuba.zip'

mapname:  conversion of  failed


In [5]:
def l1_distance(x, y):
    return abs(x - y)

def l2_distance(x, y):
    return (x - y) ** 2

def pixel_wise_matching_l1(left_img, right_img, disparity_range, save_result = True):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_value = 255

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_value

        for j in range(disparity_range):
            cost = max_value if (x - j) < 0 else l1_distance(int(left[y, x]), int(right[y, x - j]))

            if cost < cost_min:
                cost_min = cost
                disparity = j

        depth[y, x]  = disparity * scale

    if save_result == True:
        print('Saving result...')
        cv2.imwrite(f'pixel_wise_l1.png', depth)
        cv2.imwrite(f'pixel_wise_l1_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done')
    return depth

def pixel_wise_matching_l2(left_img, right_img, disparity_range, save_result = True):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_value = 255

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_value

        for j in range(disparity_range):
            cost = max_value if (x - j) < 0 else l2_distance(int(left[y, x]), int(right[y, x - j]))

            if cost < cost_min:
                cost_min = cost
                disparity = j

        depth[y, x]  = disparity * scale

    if save_result == True:
        print('Saving result...')
        cv2.imwrite(f'pixel_wise_l2.png', depth)
        cv2.imwrite(f'pixel_wise_l2_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print('Done')
    return depth

In [6]:
left_img_path = 'left.png'
right_img_path = 'right.png'
disparity_range = 16

pixel_wise_result_l1 = pixel_wise_matching_l1(left_img_path,
                               right_img_path,
                               disparity_range,
                               save_result=True)

pixel_wise_result_l2 = pixel_wise_matching_l2(left_img_path,
                               right_img_path,
                               disparity_range,
                               save_result=True)

Saving result...
Done
Saving result...
Done


In [14]:
def window_based_matching_l1(left_img, right_img, disparity_range, kernel_size = 5, save_result = True):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    depth = np.zeros((height, width), np.uint8)

    kernel_half = int((kernel_size - 1) / 2)
    scale = 3
    max_value = 255 * 9

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = 65534

            for j in range(disparity_range):
                total = 0
                value = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        value = max_value
                        if (x + u - j) >= 0:
                            value = l1_distance(int(left[y+v, x+u]), int(right[y+v, (x + u) - j]))
                        total += value

                    if total < cost_min:
                        cost_min = total
                        disparity = j

                depth[y, x] = disparity * scale

            if save_result == True:
                print('Saving results...')
                cv2.imwrite(f'window_based_l1.png', depth)
                cv2.imwrite(f'window_based_l1_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

            print('Done')
            return depth

def window_based_matching_l2(left_img, right_img, disparity_range, kernel_size = 5, save_result = True):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    depth = np.zeros((height, width), np.uint8)

    kernel_half = int((kernel_size - 1) / 2)
    scale = 3
    max_value = 255 * 9

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = 65534

            for j in range(disparity_range):
                total = 0
                value = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        value = max_value
                        if (x + u - j) >= 0:
                            value = l2_distance(int(left[y+v, x+u]), int(right[y+v, (x + u) - j]))
                        total += value

                    if total < cost_min:
                        cost_min = total
                        disparity = j

                depth[y, x] = disparity * scale

            if save_result == True:
                print('Saving results...')
                cv2.imwrite(f'window_based_l2.png', depth)
                cv2.imwrite(f'window_based_l2_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

            print('Done')
            return depth

In [15]:
left_img_path = '/content/Aloe/Aloe_left_1.png'
right_img_path = '/content/Aloe/Aloe_right_1.png'
disparity_range = 64
kernel_size = 3

window_based_result = window_based_matching_l1(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size = kernel_size,
    save_result = True
)

window_based_result = window_based_matching_l2(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size = kernel_size,
    save_result = True
)

Saving results...
Done
Saving results...
Done


In [16]:
def cosine_similarity(x, y):
    numerator = np.dot(x, y)
    denominator = np.linalg.norm(x) * np.linalg.norm(y)
    return numerator / denominator

def window_based_matching(left_img, right_img, disparity_range, kernel_size = 5, save_result = True):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    depth = np.zeros((height, width), np.uint8)

    kernel_half = int((kernel_size - 1) / 2)
    scale = 3

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_optimal = -1

            for j in range(disparity_range):
                d = x - j
                cost = -1

                for j in range(disparity_range):
                    d = x - j
                    cost = -1
                    if (d - kernel_half) > 0:
                        wp = left[(y - kernel_half):(y + kernel_half + 1), (x - kernel_half):(x + kernel_half + 1)]
                        wqd = right[(y - kernel_half):(y + kernel_half + 1), (d - kernel_half):(d + kernel_half + 1)]
                        cost = cosine_similarity(wp.flatten(), wqd.flatten())

                    if cost > cost_optimal:
                        cost_optimal = cost
                        disparity = j

                depth[y, x] = disparity * scale

            if save_result == True:
                print('Saving results...')
                cv2.imwrite(f'window_based_cosine_similarity.png', depth)
                cv2.imwrite(f'window_based_cosine_similarity_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

            print('Done')
            return depth